In [1]:
import os
import numpy as np
import pandas as pd
from model import MRIA_IO as MRIA
from table import io_basic
from tqdm import tqdm
data_path = os.path.join('..','Data')

In [2]:
import pyomo

In [3]:
output_path = os.path.join('..','results')

"""Specify disruption"""
output_dir = os.path.join(
    output_path,
    'agriculture_failure')

### Get regional mapper dictionary, to make everything compatible

In [4]:
reg_mapper = pd.read_excel(os.path.join(data_path,'economic_IO_tables','input','sh_cou_06_16.xls'),
                          sheet_name='reg_mapper',header=None)
reg_mapper = dict(zip(reg_mapper[0], reg_mapper[1]))

### Create input-output database file to be used with optimization model

In [5]:
%%time
""" Specify file path """
filepath = os.path.join(data_path, 'economic_IO_tables','output', 'IO_ARGENTINA.xlsx')

regions = ['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro',
       'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago del Estero', 'Tucuman', 'Tierra del Fuego']

regions = [x.replace(' ','_') for x in regions]

"""Create data input"""
DATA = io_basic('Argentina', filepath,regions)
DATA.prep_data()

Wall time: 14.4 s


### RUN MRIA model for all baseline

In [6]:
%%time
"""Run model and create some output"""
output_in = pd.DataFrame()

disr_dict_fd = {}
disr_dict_sup = {}

for solver in ['ipopt']:
    """Create model"""
    MRIA_RUN = MRIA(DATA.name, DATA.regions, DATA.sectors, list_fd_cats=['FD'])

    """Define sets and alias"""
    # CREATE SETS
    MRIA_RUN.create_sets()

    # CREATE ALIAS
    MRIA_RUN.create_alias()

    """ Define tables and parameters"""
    MRIA_RUN.baseline_data(DATA, disr_dict_sup, disr_dict_fd)
    MRIA_RUN.impact_data(DATA, disr_dict_sup, disr_dict_fd)

    status = MRIA_RUN.run_impactmodel(solver=solver)
    print(pd.Series(MRIA_RUN.X.get_values()).sum())

#MRIA_RUN.run_basemodel()

"""Get base line values"""
output_in['x_in'] = pd.Series(MRIA_RUN.X.get_values())
output_in.index.names = ['region', 'sector']

    ipopt


ApplicationError: No executable found for solver 'ipopt'

In [7]:
output_in['x_in'].sum()

KeyError: 'x_in'

In [ ]:
%%time
"""Run model and create some output"""
disr_dict_fd = {}
collect_outputs = {}

total_losses = {}

sum_disr = 0
event = 'agriculture shock'
for solver in ['ipopt']:
#    for event in tqdm(list(reg_mapper.values())):

    output = output_in.copy()
    """Run model and create some output"""
    disr_dict_fd = {}
    disr_dict_sup = {('Cordoba', 'A'): 1-0.083,
                     ('Santa_Fe', 'A'): 1-0.083,
                    ('Buenos_Aires', 'A'): 1-0.083}

    """Create model"""
    MRIA_RUN = MRIA(DATA.name, DATA.regions, DATA.sectors, list_fd_cats=['FD'])

    """Define sets and alias"""
    # CREATE SETS
    MRIA_RUN.create_sets()

    # CREATE ALIAS
    MRIA_RUN.create_alias()
    """ Define tables and parameters"""
    MRIA_RUN.baseline_data(DATA, disr_dict_sup, disr_dict_fd)
    MRIA_RUN.impact_data(DATA, disr_dict_sup, disr_dict_fd)

    """Get direct losses """
    disrupt = pd.DataFrame.from_dict(disr_dict_sup, orient='index')
    disrupt.reset_index(inplace=True)
    disrupt[['region', 'sector']] = disrupt['index'].apply(pd.Series)
    disrupt.drop('index', axis=1, inplace=True)
    disrupt = 1 - disrupt.groupby(['region', 'sector']).sum()
    disrupt.columns = ['shock']

    output['dir_losses'] = (disrupt['shock']*output['x_in']).fillna(0)*-1

    status = MRIA_RUN.run_impactmodel(output='print',solver=solver,tol=1e-6, DisWeight=1.75, RatWeight=2.0)
    print(status)

    output['x_out'] = pd.Series(MRIA_RUN.X.get_values())
    output['total_losses'] = (output['x_out'] - output['x_in'])
    output['ind_losses'] = (output['total_losses'] - output['dir_losses'])
    output = output.sort_index(level=[0,1])
    output.to_csv(os.path.join(output_dir, '{}.csv'.format(event)))

    prov_impact = output.groupby(level=0, axis=0).sum()[['dir_losses','total_losses','ind_losses']]
    collect_outputs[solver] = prov_impact

    total_losses_sum = (output['total_losses'].sum().sum())/1000
    total_losses[solver] = total_losses_sum

    print('{} results in {} billion pesos losses for {}'.format(event,total_losses_sum,solver))
    print('{} rationing'.format(sum(pd.Series(MRIA_RUN.Rat.get_values()).sort_values(ascending=False))/1000))

In [ ]:
output.groupby(level=1).sum().sort_values('total_losses').to_csv('D:\Dropbox\OIA\Argentina\Data\economic_IO_tables\output\losses.csv')

In [ ]:
(output.sum()['x_out']-output.sum()['x_in'])/output.sum()['x_in']*100

In [ ]:
output.to_csv('D:\Dropbox\OIA\Argentina\Data\economic_IO_tables\output\losses_reg.csv')

In [ ]:
total_impact = output.groupby(level=1).sum().sort_values('total_losses')

In [ ]:
total_impact.to_csv('D:\Dropbox\OIA\Argentina\Data\economic_IO_tables\output\losses.csv')

In [ ]:
total_impact['rel_losses'] = ((total_impact.x_out-total_impact.x_in)/total_impact.x_in)*100

In [ ]:
pd.Series(MRIA_RUN.Rat.get_values()).sort_values(ascending=False)/1e3

In [ ]:
pd.Series(MRIA_RUN.A_matrix.values())